In [6]:
%load_ext autoreload
%autoreload 2
import spacy
import scispacy
from scispacy.linking import EntityLinker
import en_core_sci_lg
from scify.nlp import *
from scify.utils.GNBR_preprocessing import get_data_and_distributions

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


## TODO
[x] fix entity shift in char offset

[x] merge entities yeah

[x] put utils in .py files. I hate those long Jupyter notebooks with their uncomposable trash python code all over the place 

[ ] Fix error in merge_file thing

[ ] can use multiple labels (since all are in entity._.kb_ents {bc5, jnlpa...}

[ ] Add more annotations as extension create product(UMLS, MESH, ontologies) X (models) annotated file from pubmed abstracts to not always load everything from scratch

[ ] slowly create multi-model NER annotated datasets to easy sample and do experiments on

[ ] look at source code for kindred, scipacy, role_pattern_nlp and adapt utils

[ ] EDA on corpus level stats and word-trees


UMLS is really big. Better to load linkers in separate cells. Sometimes crashes or similar on my Macbook Air

In [7]:
# This line takes a while, because we have to download ~1GB of data
# and load a large JSON file (the knowledge base). Be patient!
# Thankfully it should be faster after the first time you use it, because
# the downloads are cached.
# NOTE: The resolve_abbreviations parameter is optional, and requires that
# the AbbreviationDetector pipe has already been added to the pipeline. Adding
# the AbbreviationDetector pipe and setting resolve_abbreviations to True means
# that linking will only be performed on the long form of abbreviations.

#UMLS is big and takes a while EVEN when CACHED it is downloading from tfidf_vectors from here idk why:
#url: /ai2-s2-scispacy/data/umls/tfidf_vectors_sparse.npz
linker_umls = EntityLinker(resolve_abbreviations=True, name="umls")

#linker_go = EntityLinker(resolve_abbreviations=True, name="go")
#linker_mesh = EntityLinker(resolve_abbreviations=True, name="mesh")

/Users/markus/opt/anaconda3/envs/markus_nlp/lib/python3.8/site-packages/sklearn/base.py:329: UserWarning: Trying to unpickle estimator TfidfTransformer from version 0.20.3 when using version 0.23.1. This might lead to breaking code or invalid results. Use at your own risk.
  warnings.warn(
/Users/markus/opt/anaconda3/envs/markus_nlp/lib/python3.8/site-packages/sklearn/base.py:329: UserWarning: Trying to unpickle estimator TfidfVectorizer from version 0.20.3 when using version 0.23.1. This might lead to breaking code or invalid results. Use at your own risk.
  warnings.warn(


In [8]:
#linkers = [EntityLinker(resolve_abbreviations=True, name=ontology) for ontology in ["umls", "go", "mesh"]] this might crash
#linkers = [linker_mesh, linker_go, linker_umls]
#[get_ontology_name(linker) for linker in linkers]

#NER_models = [en_ner_bc5cdr_md, en_ner_craft_md, en_ner_bionlp13cg_md, en_ner_jnlpba_md]
#nlps = [model.load() for model in NER_models]

In [9]:
#Specialized NER models trained on specialized corpus
import en_ner_bc5cdr_md
import en_ner_craft_md
import en_ner_bionlp13cg_md
import en_ner_jnlpba_md

In [10]:
text="Spinal and bulbar muscular atrophy (SBMA) is an \
           inherited motor neuron disease caused by the expansion \
           of a polyglutamine tract within the androgen receptor (AR). \
           SBMA can be caused by this easily."

I'm loading models here in every cell (instead of mapping them) separately because it sometimes crashes...

In [11]:
base_nlp = spacy.load("en_core_sci_lg")

In [12]:
nlp1 = spacy.load("en_ner_bc5cdr_md")

In [13]:
nlp2 = spacy.load("en_ner_craft_md") #craft has loading issues? buggy....

In [14]:
nlp3 = en_ner_bionlp13cg_md.load()

In [15]:
nlp4 = en_ner_jnlpba_md.load()

In [16]:
NER_nlps = [nlp1, nlp2, nlp3, nlp4]

In [17]:
nlps = add_pipes_mutative(NER_nlps, linker_umls)

In [18]:
base_nlp.pipeline, nlps[3].pipeline, nlps[0].meta["name"]

([('tagger', <spacy.pipeline.pipes.Tagger at 0x1aee18a60>),
  ('parser', <spacy.pipeline.pipes.DependencyParser at 0x1218c7a00>),
  ('ner', <spacy.pipeline.pipes.EntityRecognizer at 0x2c2ef8e20>)],
 [('tagger', <spacy.pipeline.pipes.Tagger at 0x3281ddee0>),
  ('parser', <spacy.pipeline.pipes.DependencyParser at 0x10a65e280>),
  ('ner', <spacy.pipeline.pipes.EntityRecognizer at 0x3447df640>),
  ('AbbreviationDetector',
   <scispacy.abbreviation.AbbreviationDetector at 0x34f8a7d00>),
  ('merge_entities', <function spacy.pipeline.functions.merge_entities(doc)>),
  ('EntityLinker', <scispacy.linking.EntityLinker at 0x122e5eac0>)],
 'ner_bc5cdr_md')

Be Very careful to overwrite Ents .... there's many attributes that are needed! I just attach an extension
(but in the end, my algorithm just looks at labels_ to determine if a sentence is worthy of pattern matching!)

1) _token the product(NER modules x mesh/go/umls)
2) Read the Text with all of them
3) Extract their labels and ids + confidence scores
4) Look at set differences -> overlap intersect?
4) Add labels to doc @ Scispacy_lg (no NER) so it has Labels (set_extension)
5) Assign highest confidence label to scispacy large

In [19]:
#linker_umls.kb.cui_to_entity["5372017"]
#D016472 is a mesh ID

In [20]:
chem_gene_in_texts = [
    "The antiarrhythmic effect of carvedilol was examined in a placebo-controlled multicenter trial , the Carvedilol Post-Infarct Survival Control in Left_Ventricular_Dysfunction -LRB- CAPRICORN -RRB- study , which enrolled 1,959 patients with reduced left ventricular systolic function after AMI , 98 % of whom were treated with an ACE inhibitor .",
   "Carvedilol improves survival of patients suffering from CHF but the effects of the drug on angiotensin-converting_enzyme -LRB- ACE -RRB- activity , renin and aldosterone are not well characterized in patients receiving an ACE inhibitor .",
"We therefore assessed the effects of introducing carvedilol and bisoprolol in a prospective manner on indices of oxidative stress -LSB- lipid hydroperoxides -LRB- LHP -RRB- -RSB- , endothelial damage -LSB- von_Willebrand factor -LRB- vWf -RRB- -RSB- , platelet activation -LRB- soluble P-selectin -RRB- and coagulation -LRB- fibrinogen -RRB- and their inter-relationships in stable outpatients with CHF in sinus rhythm ."
]

In [21]:
%time
docs = get_merged_docs_for_texts(chem_gene_in_texts, base_nlp, nlps)

CPU times: user 3 µs, sys: 1e+03 ns, total: 4 µs
Wall time: 6.91 µs
Merging Named Entities (Chems, Gene, Organism etc.). Later docs in the pipeline overwrite entities from earlier ones


In [22]:
visualise_doc(docs[0], compact=True)

In [23]:
from scify.nlp import construct_pattern, prep_pattern

In [24]:
construct_pattern(prep_pattern("kinase|compound|START_ENTITY RAFTK|nsubj|kinase RAFTK|dep|is is|nsubj|kinase kinase|nmod|activation activation|nmod|END_ENTITY"))

In [25]:
sent1 = """In this study , we report that the related adhesion focal tyrosine kinase RAFTK , is an upstream kinase in beta1 integrin mediated activation of Akt."""

In [26]:
visualise_doc(base_nlp(sent1))

In [27]:
#get_non_zero support for one of the 30 themes only
#dependency paths + support
#Maybe merge dependency paths
#distributions

#it's very big but looks like this
distributions_kv_example = {"kinases|compound|START_ENTITY participate|nsubj|kinases participate|nmod|END_ENTITY": {'A+': 2.0,
  'A+.ind': 0.0,
  'A-': 0.0,
  'A-.ind': 0.0,
  'B': 0.0,
  'B.ind': 0.0,
  'E+': 0.0,
  'E+.ind': 0.0,
  'E-': 0.0,
  'E-.ind': 0.0,
  'E': 0.0,
  'E.ind': 0.0,
  'N': 0.0,
  'N.ind': 0.0,
  'O': 2.0,
  'O.ind': 0.0,
  'K': 0.0,
  'K.ind': 0.0,
  'Z': 0.0,
  'Z.ind': 0.0}}

In [28]:
from scify.nlp import rgetattr, rsetattr
linker_umls.kb.cui_to_entity["C1413931"]

CUI: C1413931, Name: ACE gene
Definition: This gene plays a role in vasoconstriction.
TUI(s): T028
Aliases (abbreviated, total: 19): 
	 ACE gene, ACE gene, ACE Gene, ACE Gene, ANGIOTENSIN I-CONVERTING ENZYME, angiotensin I converting enzyme, KININASE II, peptidyl-dipeptidase A, ACE, ACE

In [30]:
[annotation["label"] 
 for ent in docs[0].ents 
 for annotation in ent._.annotated]

['CHEMICAL',
 'SIMPLE_CHEMICAL',
 'SIMPLE_CHEMICAL',
 'DNA',
 'ORGANISM',
 'MULTI-TISSUE_STRUCTURE',
 'CHEMICAL',
 'GENE_OR_GENE_PRODUCT',
 'CHEBI']

In [32]:
[doc for doc in docs if doc_has_entity_labels(doc,  [["CHEMICAL", "SIMPLE_CHEMICAL"], ["GENE", "GENE_OR_GENE_PRODUCT"]])]

[The antiarrhythmic effect of carvedilol was examined in a placebo-controlled multicenter trial , the Carvedilol Post-Infarct Survival Control in Left_Ventricular_Dysfunction -LRB- CAPRICORN -RRB- study , which enrolled 1,959 patients with reduced left ventricular systolic function after AMI , 98 % of whom were treated with an ACE inhibitor .,
 Carvedilol improves survival of patients suffering from CHF but the effects of the drug on angiotensin-converting_enzyme -LRB- ACE -RRB- activity , renin and aldosterone are not well characterized in patients receiving an ACE inhibitor .]

In [33]:
import spacy #from spacy.tokens import DocBin
import pickle
doc_data = pickle.dumps(docs) #only pickle lists of docs so it only includes their shared vocabulary
with open('../data/pubmed_annotated.pickle', 'wb') as f:
    pickle.dump(doc_data, f)
print(len(doc_data)) #that's a lot for 3 sentences

759919990


In [34]:
doc_data2 = pickle.dumps(docs[1:]) #only pickle lists of docs so it only includes their shared vocabulary
with open('../data/pubmed_annotated.pickle', 'wb') as f:
    pickle.dump(doc_data, f)
print(len(doc_data2)) #ok but it's only the vocab and scales well!

759893880


In [35]:
data, distributions = get_data_and_distributions()

['A+', 'A+.ind', 'A-', 'A-.ind', 'B', 'B.ind', 'E+', 'E+.ind', 'E-', 'E-.ind', 'E', 'E.ind', 'N', 'N.ind', 'O', 'O.ind', 'K', 'K.ind', 'Z', 'Z.ind'] headers 
        chemical-gene
(A+) agonism, activation
(A-) antagonism, blocking
(B) binding, ligand (esp. receptors)
(E+) increases expression/production
(E-) decreases expression/production
(E) affects expression/production (neutral)
(N) inhibits
        


In [36]:
sents = [dat["sent"] for dat in data]


In [37]:
len(sents)
sent_sample = sents[:500]

In [38]:
docs = get_merged_docs_for_texts(sent_sample, base_nlp, nlps)

Merging Named Entities (Chems, Gene, Organism etc.). Later docs in the pipeline overwrite entities from earlier ones


In [39]:
wanted_sents = [doc for doc in docs if doc_has_entity_labels(doc,  [["CHEMICAL", "SIMPLE_CHEMICAL"], ["GENE", "GENE_OR_GENE_PRODUCT"]])]

Let's see how well our NER modules work. Theoretically every sentence in the data should have at least one chemical and one gene entity in it

Ok that's 40% coverage....that's kinda bad

In [40]:
len(wanted_sents), wanted_sents[200]


(207,
 MAIN OUTCOMES MEASURES : Progesterone , 17-hydroxyprogesterone , 11-deoxycorticosterone , corticosterone , aldosterone , 11-deoxycortisol , cortisol , cortisone , dehydroepiandrosterone_sulfate -LRB- DHEAS -RRB- , androstenedione , T , dihydrotestosterone , insulin resistance index of the homeostasis model assessment , and blood pressure were measured .)

Let's check for patterns


In [41]:
patterns = [d["dep"] for d in data[:500]]

constructed = [construct_pattern(pattern) for pattern in patterns]

len([con for con in constructed if con != None])

430

I can live with that ratio 430/500 (maybe errors not evenly distributed)

In [42]:
import os
os.listdir("../data/pubmed")

['sample_pubmed20n0004.xml',
 'draw.io-13.5.1.dmg',
 '.DS_Store',
 'pubmed20n0008.xml.gz',
 'pubmed20n0004.xml.gz',
 'pubmed20n0004.xml',
 'pubmed20n0010.xml',
 'pubmed20n0001.xml',
 'pubmed20n0010.xml.gz',
 'LB4-Linguamatics-Brochure-Life-Science-overview.pdf',
 'J_Entrez.txt',
 'Zusammenfassung.pdf',
 'pubmed20n0008.xml',
 '1908.10784.pdf',
 'pubmed20n0001.xml.gz',
 'pubmed20n1015.xml',
 'sample_pubmed_1.xml']

In [43]:
from pubmed_parser import parse_medline_xml
pubmed_abstracts = parse_medline_xml("../data/pubmed/pubmed20n1015.xml")
abstr = [article["abstract"] for article in pubmed_abstracts]

In [72]:
%time
#already in pickle
#docs_pubmed = get_merged_docs_for_texts(abstr, base_nlp, nlps)

CPU times: user 3 µs, sys: 0 ns, total: 3 µs
Wall time: 6.91 µs
Merging Named Entities (Chems, Gene, Organism etc.). Later docs in the pipeline overwrite entities from earlier ones
index : 18 --- error for doc: Renal oste
index : 21 --- error for doc: Background
index : 32 --- error for doc: Mesenchyma
index : 42 --- error for doc: Introducti
index : 48 --- error for doc: Introducti
index : 48 --- error for doc: Introducti
index : 62 --- error for doc: Background
index : 62 --- error for doc: Background
index : 81 --- error for doc: Background
index : 87 --- error for doc: Mutations 
index : 89 --- error for doc: Correlatio
index : 96 --- error for doc: Objective:
index : 99 --- error for doc: Objective:
index : 99 --- error for doc: Objective:
index : 99 --- error for doc: Objective:
index : 112 --- error for doc: Arabinoman
index : 130 --- error for doc: Background
index : 138 --- error for doc: During pre
index : 139 --- error for doc: Intraventr
index : 140 --- error for doc: We pr

In [133]:
#import spacy #from spacy.tokens import DocBin
import pickle
doc_data = pickle.dumps(docs_pubmed) #only pickle lists of docs so it only includes their shared vocabulary
with open('../data/pubmed_annotated.pickle', 'wb') as f:
    pickle.dump(doc_data, f)
print(len(doc_data)) #that's a lot for 3 sentences

846955806


In [121]:
[sent.ents for sent in dd.sents]
from typeguard import typechecked

from scify.nlp import *

[doc_has_entity_labels(sent, [["EE", "EE"], ["rr", "rr"]]) for sent in dd.sents]

[False, False]

In [74]:
wanted_abstr = [doc for doc in docs_pubmed if doc_has_entity_labels(doc,  [["CHEMICAL", "SIMPLE_CHEMICAL"], ["GENE", "GENE_OR_GENE_PRODUCT"]])]

In [129]:

len([sent for abstract in wanted_abstr for sent in abstract.sents ]), len([sent for abstract in wanted_abstr for sent in abstract.sents if doc_has_entity_labels(sent,  [["CHEMICAL", "SIMPLE_CHEMICAL"], ["GENE", "GENE_OR_GENE_PRODUCT"]])])

(3022, 491)

In [135]:
pubmed_sents =[sent for abstract in wanted_abstr for sent in abstract.sents if doc_has_entity_labels(sent,  [["CHEMICAL", "SIMPLE_CHEMICAL"], ["GENE", "GENE_OR_GENE_PRODUCT"]])]
pubmed_sents[22]

Additionally, the ligand of PPAR-γ was incubated with epithelial cells to interfere with the effects of lipopolysaccharide (LPS) or rhHMGB1 to explore the effect on expression of HMGB1 and EMT markers.

In [136]:
visualise_doc(pubmed_sents[22])

In [143]:
show_noun_chunks(pubmed_sents[22])

Chunk          .root      root.dep_     .root.head            
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
the ligand     ligand     nsubjpass     incubated             
(LPS           LPS        appos         lipopolysaccharide    
the effect     effect     dobj          explore               


[autoreload of scify.nlp failed: Traceback (most recent call last):
  File "/Users/markus/opt/anaconda3/envs/markus_nlp/lib/python3.8/site-packages/IPython/extensions/autoreload.py", line 245, in check
    superreload(m, reload, self.old_objects)
  File "/Users/markus/opt/anaconda3/envs/markus_nlp/lib/python3.8/site-packages/IPython/extensions/autoreload.py", line 410, in superreload
    update_generic(old_obj, new_obj)
  File "/Users/markus/opt/anaconda3/envs/markus_nlp/lib/python3.8/site-packages/IPython/extensions/autoreload.py", line 347, in update_generic
    update(a, b)
  File "/Users/markus/opt/anaconda3/envs/markus_nlp/lib/python3.8/site-packages/IPython/extensions/autoreload.py", line 266, in update_function
    setattr(old, name, getattr(new, name))
ValueError: get_entity_diff() requires a code object with 0 free vars, not 3
]


In [ ]:
def get_shortest_dep_path(ent_a, ent_b):
    #networkx

def pattern_from_shortest_dep_path():
    5
    
def extract_pattern_between_entities(doc, ent_a, ent_b):
    4
    


In [157]:
from scify.networks import get_edges
import networkx as nx
get_edges(pubmed_sents[22])


AttributeError: module 'networkx' has no attribute 'g'

In [209]:
doc = docs_pubmed[22]



In [207]:
d

The goal of this study was to examine the effect of lipid emulsion on the vasodilation induced in isolated endothelium-denuded rat aortae by a toxic dose of amlodipine. We examined the effects of lipid emulsion and verapamil on amlodipine-induced vasodilation. We also examined the effects of a mixture of lipid emulsion and amlodipine, as well as the centrifuged aqueous extract (CAE) obtained by ultracentrifuging such a mixture and then removing the upper lipid layer, on amlodipine-induced vasodilation. The effect of lipid emulsion on the amlodipine concentration was examined. Lipid emulsion attenuated amlodipine-induced vasodilation in isolated aortae. Both CAE and lipid emulsion containing amlodipine inhibited amlodipine-induced vasodilation. However, there was no significant difference in amlodipine-induced vasodilation between aortae treated with CAE and those treated with lipid emulsion containing amlodipine. Verapamil inhibited amlodipine-induced vasodilation. Lipid emulsion decre

In [175]:
from itertools import product

[* product(["Yolo", "Fredo", "checko"], ["fatty", "fagg"])]
tokens_lookup = {tok.text:tok for tok in doc}

In [ ]:
#TODO
#token -> char overlap in merge_docs
#document level matching %
#sentence level matching %
#performance?

#get_shortest_path between ents
#find pattern matches
#dependency pattern + pattern matcher on GNBR --> relation extraction precision?


In [202]:
for token in doc[:25]:
    print(token, token.i, token.ent_iob)
print([noun for noun in doc[:25].noun_chunks])

The 0 2
goal 1 3
of 2 2
this 3 2
study 4 3
was 5 2
to 6 2
examine 7 2
the 8 2
effect 9 3
of 10 2
lipid 11 3
emulsion 12 2
on 13 2
the 14 2
vasodilation 15 3
induced 16 3
in 17 2
isolated 18 3
endothelium-denuded 19 3
rat 20 3
aortae 21 1
by 22 2
a 23 2
toxic 24 3
[The goal, the effect]


In [214]:
from spacy.tokens import Span
def semantic_spans_from_doc(doc):
    """returns a list of all spans eg. [token, token, entitiy (spanning more tokens), token]
    This is so you can iterate through it under one span API
    
    iobs
    -3 begins
    -2 outside
    -1 inside
    -0 no tag of
    ...entity
    """
    "3 1 1"
    "3 2 3 1"
    "2 3 1"
    spans = []
    current_span = []
    for idx, token in enumerate(doc):
        iob = token.ent_iob
        if (iob != 1):
            spans.append(Span)
            current_span.append()
        
        
    return spans, doc

ValueError: [E105] The Doc.print_tree() method is now deprecated. Please use Doc.to_json() instead or write your own function.

In [255]:
a = [1 ,2 ,3]

a.pop(0), a

(1, [2, 3])

In [ ]:
import copy
copy.deepcopy(doc)

In [ ]:
ent_idxs = [ent.start for ent in doc.ents]
non_ent_idxs = [token.i for token in doc if (token.ent_iob == 2 or token.ent_iob == 0)]


In [ ]:
sort(ent_idxs + non_ent_idx)


In [ ]:

spans =[]
ent_idxs = [ent.start for ent in doc.ents]
ents = doc.ents.copy()
for idx, token in enumerate(doc):
    if idx in ents_idxs:
        spans.append(doc[idx])
        
    


In [256]:
[ent.start for ent in doc.ents],[token.i for token in doc if (token.ent_iob == 2 or token.ent_iob == 0)]


([3, 5, 8, 15, 18, 21, 32, 41, 51, 54],
 [0,
  1,
  2,
  4,
  6,
  7,
  9,
  10,
  11,
  12,
  13,
  14,
  16,
  17,
  19,
  20,
  24,
  25,
  26,
  27,
  28,
  29,
  30,
  31,
  34,
  35,
  36,
  37,
  38,
  39,
  40,
  43,
  44,
  45,
  46,
  47,
  48,
  49,
  50,
  52,
  53,
  55,
  56])

In [231]:
[token.i for token in doc[:25] if token.ent_iob == 3],[token.i for token in doc[:25] if token.ent_iob != 2]

([3, 5, 8, 15, 18, 21], [3, 5, 8, 15, 18, 21, 22, 23])

In [232]:
#uniform 
#lowest_common_ancestor span

In [233]:
#valid pattern
construct_pattern('|appos|START_ENTITY sensitivity|nmod|END_ENTITY') or []

[]

In [234]:
#invalid pattern
construct_pattern('sensitivity|nmod|END_ENTITY sensitivity|appos|START_ENTITY')

[{'SPEC': {'NODE_NAME': 'sensitivity'}, 'PATTERN': {'LEMMA': 'sensitivity'}},
 {'SPEC': {'NODE_NAME': 'END_ENTITY',
   'NBOR_RELOP': '>',
   'NBOR_NAME': 'sensitivity'},
  'PATTERN': {'DEP': 'nmod', 'POS': 'NOUN'}},
 {'SPEC': {'NODE_NAME': 'START_ENTITY',
   'NBOR_RELOP': '>',
   'NBOR_NAME': 'sensitivity'},
  'PATTERN': {'DEP': 'appos', 'POS': 'NOUN'}}]

In [235]:
from scify.networks import get_edges

TODO
1) isEntity in Sentence?
2) Shortest Path -> SP
3) compare with theme in GNBR